In [ ]:
import pandas as pd
import numpy as np
import rasterio
import os
from osgeo import gdal, gdalconst

In [85]:
ndvi_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Africa/'
ndvi_comp_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Compressed/'
ndvi_file = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Compressed/c_data.2002.071.tif'
ndvi_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EA/'
ndvi_extra = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Extra/'

lst_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Original/'
lst_comp_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Compressed/'
lst_file = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Original/data.2002.071.tif'
lst_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/EA/'
lst_extra = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Extra/'

precip_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Original/'
precip_comp_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Compressed/'
precip_file = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Original/chirps-v2.0.2002.07.1.tif'
precip_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/EA/'
precip_extra = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Extra/'

In [86]:
lc_2016 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/ESACCI_2016/original/'
lc_ea_2016 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/ESACCI_2016/EA/'

In [ ]:
def resample(in_dir, ref_file, out_dir, prefix):
    
    '''
    This function resamples rasters in a directory given a reference file's pixel size, projection and extent and creates new files in an out directory.
    '''
    
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
    for filename in files:
        
        base = os.path.splitext(filename)[0]
        print(base)
        inputfile = in_dir + filename
        input = gdal.Open(inputfile, gdalconst.GA_ReadOnly)
        inputProj = input.GetProjection()
        inputTrans = input.GetGeoTransform()

        referencefile = ref_file
        reference = gdal.Open(referencefile, gdalconst.GA_ReadOnly)
        referenceProj = reference.GetProjection()
        referenceTrans = reference.GetGeoTransform()
        bandreference = reference.GetRasterBand(1)
        
        
#         x = reference.RasterXSize 
#         y = reference.RasterYSize
        
        x = 583
        y = 674


        outputfile = out_dir + prefix + base + '.tif' #change output file name for each variable
        driver= gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfile,x,y,1,bandreference.DataType)
        output.SetGeoTransform((22, 0.05, 0.0, 22, 0.0, -0.05)) #edited here to get certain extent 
        output.SetProjection(referenceProj)
        
        gdal.ReprojectImage(input,output,inputProj,referenceProj,gdalconst.GRA_Bilinear)

        del output

In [80]:
resample(lc_2016, ndvi_file, lc_ea_2016, 'EA_')

ESACCI_LC_2016_005


In [88]:
#set all 0's to nan in NDVI files
#set all -9999 to nan in precip and lst files

def reassign_mask_npnan(in_dir):

    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
        
    for filename in files:
        with rasterio.open(in_dir+filename, 'r+') as ds:
            print(filename)
            a = ds.read()# read all raster values
            a[a == -9999 ] = np.nan  
            ds.write(a)

In [ ]:
reassign_mask_npnan(precip_ea)